In [2]:
import torch
import torchvision
from tvm import relay
# Load ShuffleNetV2
shufflenet = torch.hub.load('pytorch/vision:v0.14.0', 'shufflenet_v2_x1_0', pretrained=True).eval()

# Example input
x_shuffle = torch.randn(1, 3, 224, 224)

# Forward pass (optional check)
y_shuffle = shufflenet(x_shuffle)
print("ShuffleNet output shape:", y_shuffle.shape)

# Trace and convert to TVM Relay
scripted_shuffle = torch.jit.trace(shufflenet, x_shuffle).eval()
mod_shuffle, params_shuffle = relay.frontend.from_pytorch(scripted_shuffle, [("input0", x_shuffle.shape)])
print("✅ ShuffleNet successfully converted to TVM Relay")




Using cache found in /home/vscode/.cache/torch/hub/pytorch_vision_v0.14.0
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ShuffleNet output shape: torch.Size([1, 1000])
✅ ShuffleNet successfully converted to TVM Relay


In [3]:
import tvm
from tvm.contrib import graph_executor

# Choose your target architecture
# Example: x86 CPU
target_str = "llvm -mtriple=x86_64-linux-gnu -mcpu=haswell"
target = tvm.target.Target(target_str)
dev = tvm.cpu(0)


print(f"Compiling ShuffleNetV2...")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_shuffle, target=target, params=params_shuffle)
# Save compiled library
lib_name = f"ShuffleNetV2_tvm.so"
lib.export_library(lib_name)
print(f"✅ ShuffleNetV2 compiled and exported as {lib_name}")


Compiling ShuffleNetV2...


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


✅ ShuffleNetV2 compiled and exported as ShuffleNetV2_tvm.so
